In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-14B-Chat-Int4", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-14B-Chat-Int4",
    device_map="auto",
    trust_remote_code=True
).eval()

In [ ]:
def normal_prompt(s):
  s = f'你是一个专业的日文到中文翻译器。你只应输出中文翻译结果而不附带其他任何解释或说明，并且可以正确处理日语中的众多语气词。当你无法理解或翻译一段输入时，请返回原文。现在，翻译以下内容：\n`{s}`'
  resp, _ = model.chat(tokenizer, s, history=None)
  return resp

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

class Body(BaseModel):
  text: str

@app.post('/trans')
async def root(body: Body):
    return normal_prompt(body.text)

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)